# Requirements

In [16]:
import pandas as pd
import os
import collections
import openpyxl


In [15]:
# Special Requirements for openpyxl

try:
        from openpyxl.cell import get_column_letter
except ImportError:
        from openpyxl.utils import get_column_letter
        from openpyxl.utils import column_index_from_string
from openpyxl import load_workbook
import openpyxl
from openpyxl import Workbook

# Functions built to filter DataFrames in diferent ways:

In [6]:
def filtrar_df_lista(df, str_columna, lista):
    
    '''
    INPUTS: 
    
    df : DataFrame to be filtered
    str_columna : column is going to be filtered
    lista : list elements to filter with (exactly equals to)
    
    OUTPUT:
    
    df filtered and reindexed
    
    '''
    
    new_df = pd.DataFrame()

    new_df = new_df.append([df[df[str_columna] == e] for e in lista])

    return new_df.reset_index(drop = True)
    

In [7]:
def col_contains_str(df, col, string):
    '''
    INPUTS: 
    
    df : DataFrame to be filtered
    col : column is going to be filtered
    string: if value comtains this string, it will appear in the result
    
    OUTPUT:
    
    DataFrame filtered and reindexed
    
    '''
    
    df = df.dropna(subset = [col])
    return df[df[col].isin([e for e in df[col] if string.lower() in e.lower()])].reset_index(drop = True)

def col_contains_any_str_list(df, col, list_str):
    '''
    INPUTS: 
    
    df : DataFrame to be filtered
    col : column is going to be filtered
    list_str : if the value contains some of the string in this list, it will appear in the result
    
    OUTPUT:
    
    DataFrame filtered and reindexed
    
    '''
    df = df.dropna(subset = [col])
    return df[df[col].isin([e for e in df[col] if any(i.lower() in e.lower() for i in list_str)])].reset_index(drop =  True)
            

# Functions built in order to export DataFrames to Excel

In [9]:
 
def adjust_columns_width(sheet):
    
    """
    For a given Excel-sheet, adjust the width of the columns depending on its own length
    
    """
    
    for column_cells in sheet.columns:
        new_column_length = max(len(str(cell.value)) for cell in column_cells)
        new_column_letter = (get_column_letter(column_cells[0].column))
        if new_column_length > 0:
            sheet.column_dimensions[new_column_letter].width = new_column_length*1.23
            
    sheet.sheet_view.zoomScale = 60
    
    




# Functions built to work with bad-structured DataFrames as a generic way

In [10]:
def diferenciar_columnas_iguales(columns):

    '''
    
    For a given DataFrame Columns with columns with the same name (for example when you import data from an Excel), a new Columns will be got adding '_n' to each col, being n a counter
    Example: [Codigo, valor, valor, valor, valor_final, valor_final] --> [Codigo, valor_0, valor_1, valor_2, valor_final_0, valor_final_1]
    
    INPUT: List with the columns of the DataFrame
    OUTPUT: List of the columns with a counter at the end
    
    '''
    
    import collections
    
    cols_repetidas = [k for k, v in dict(collections.Counter(columns)).items() if v > 1]

    for c_r in cols_repetidas:
        cont = 0
        for c in range(len(columns)):
            if columns[c] == c_r:
                columns[c] = columns[c] + '_' + str(cont)
                cont += 1

    return columns


# Functions built with the goal of execute automatically some manual tasks

In [13]:
def split_col_by_unique_values(df, column, name_output_directory):
    
        
    '''
    
    For a given DataFrame and Column, the output is in a folder, the same DataFrame splited in diferent Excels Files (One for each unique Value)
    The task in Excel would be: 1) Having a column with 8 diferent values
                                2) Filter by the first value, copy, paste and save
                                3) Same for the remaining values
                                4) the result are 8 Excels
    
    INPUTS: 
    
    df : DataFrame to split
    column : col to be split
    name_output_directory : name of the folder where the splitted Excels will be located (If not exists, it is created below this script)
    
    OUTPUT:
    
    Splitted Excels
    
    '''
    import pandas as pd
    import os 
    
    try:
        os.mkdir(str(name_output_directory))
    except:
        1
    
    df[column] = df[column].fillna('EMPTY')
    
    for value in [e.replace(' ', '_') for e in df[column].unique()]:

        locals()[value] = df[(df[column] == value.replace('_', ' '))]
        
        try:     
            locals()[value].to_excel(name_output_directory + '/' + str(value) + '.xlsx', index = False)
        except:  
            continue
            
            